In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import random

In [8]:
file_path="/content/drive/MyDrive/Research_dataset/Copy of test_case_reduction_large(1).xlsx"
xls = pd.ExcelFile(file_path)

In [9]:
df = pd.read_excel(xls, sheet_name="Sheet1")

In [10]:
df['Normalized_Coverage'] = df['Code Coverage (%)'] / df['Code Coverage (%)'].max()
df['Normalized_Faults'] = df['Defects Found'] / df['Defects Found'].max()
df['Normalized_Time'] = df['Execution Time (sec)'] / df['Execution Time (sec)'].max()

In [11]:
class Particle:
    def __init__(self, num_tests):
        self.position = np.random.choice([0, 1], size=num_tests)  # Binary representation
        self.velocity = np.random.uniform(-1, 1, num_tests)  # Random velocity initialization
        self.best_position = np.copy(self.position)
        self.best_fitness = -1  # Higher is better

    def update_velocity(self, global_best_position, w=0.5, c1=1.5, c2=1.5):
        inertia = w * self.velocity
        cognitive = c1 * random.random() * (self.best_position - self.position)
        social = c2 * random.random() * (global_best_position - self.position)
        self.velocity = inertia + cognitive + social

    def update_position(self):
        sigmoid = 1 / (1 + np.exp(-self.velocity))
        self.position = np.where(np.random.rand(len(self.position)) < sigmoid, 1, 0)


    def evaluate_fitness(self, df, alpha=0.5, beta=0.3, gamma=0.2):
        """
        Fitness function that optimizes:
        - High Code Coverage
        - High Fault Detection
        - Low Execution Time
        """
        selected = self.position == 1
        if np.sum(selected) == 0:  # Avoid division by zero
            return 0

        coverage = np.sum(df['Normalized_Coverage'][selected])
        faults = np.sum(df['Normalized_Faults'][selected])
        execution_time = np.sum(df['Normalized_Time'][selected])

        # We want to maximize coverage & fault detection, minimize execution time
        fitness = (alpha * coverage + beta * faults) - (gamma * execution_time)
        return fitness


In [12]:
# PSO Optimization function
def pso_test_case_reduction(df, num_particles=30, max_iter=100):
    num_tests = len(df)
    particles = [Particle(num_tests) for _ in range(num_particles)]

    global_best_position = np.zeros(num_tests)
    global_best_fitness = -1

    for _ in range(max_iter):
        for particle in particles:
            fitness = particle.evaluate_fitness(df)

            # Update personal best
            if fitness > particle.best_fitness:
                particle.best_fitness = fitness
                particle.best_position = np.copy(particle.position)

            # Update global best
            if fitness > global_best_fitness:
                global_best_fitness = fitness
                global_best_position = np.copy(particle.position)

        # Update velocity and position of all particles
        for particle in particles:
            particle.update_velocity(global_best_position)
            particle.update_position()

    # Return the reduced test cases
    return df[global_best_position == 1]

# Run the optimization
optimal_test_cases = pso_test_case_reduction(df)

# Display the reduced test cases
print("Reduced Test Cases:")
print(optimal_test_cases[['Test Case ID', 'Execution Time (sec)', 'Code Coverage (%)', 'Defects Found', 'Priority']])

# Save the reduced test cases to an Excel file
optimal_test_cases.to_excel("reduced_test_cases.xlsx", index=False)
print("Reduced test cases saved successfully!")


Reduced Test Cases:
    Test Case ID  Execution Time (sec)  Code Coverage (%)  Defects Found  \
0        TC_0001                  3.77                 87              4   
1        TC_0002                  3.29                 75              1   
3        TC_0004                  3.03                 76              4   
6        TC_0007                  3.22                 83              3   
7        TC_0008                  2.99                 91              1   
..           ...                   ...                ...            ...   
988      TC_0989                  3.80                 79              1   
991      TC_0992                  3.45                 93              4   
994      TC_0995                  1.67                 97              4   
995      TC_0996                  3.14                 95              4   
997      TC_0998                  3.13                 80              0   

    Priority  
0       High  
1       High  
3     Medium  
6      